In [1]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
Ravdess = "ravdess-emotional-speech-audio/"

Crema = "cremad/AudioWAV/"

Tess = "toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/"

Savee = "surrey-audiovisual-expressed-emotion-savee/ALL/"
print(Ravdess)
print(Tess)
print(Savee)

ravdess-emotional-speech-audio/
toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/
surrey-audiovisual-expressed-emotion-savee/ALL/


In [3]:
#promain
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for directory in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(os.path.join(Ravdess, directory))
    for file in actor:
        
        
        part = file.split('.')[0]
        
        part = part.split('-')
        

        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + directory +'/' + file)
        

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
1,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
2,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
3,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
4,calm,ravdess-emotional-speech-audio/Actor_01/03-01-...


In [4]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,cremad/AudioWAV/1001_DFA_ANG_XX.wav
1,disgust,cremad/AudioWAV/1001_DFA_DIS_XX.wav
2,fear,cremad/AudioWAV/1001_DFA_FEA_XX.wav
3,happy,cremad/AudioWAV/1001_DFA_HAP_XX.wav
4,neutral,cremad/AudioWAV/1001_DFA_NEU_XX.wav


In [5]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for directory in tess_directory_list:
    directories = os.listdir(os.path.join(Tess, directory))
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + directory +'/'+ file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,angry,toronto-emotional-speech-set-tess/TESS Toronto...
1,angry,toronto-emotional-speech-set-tess/TESS Toronto...
2,angry,toronto-emotional-speech-set-tess/TESS Toronto...
3,angry,toronto-emotional-speech-set-tess/TESS Toronto...
4,angry,toronto-emotional-speech-set-tess/TESS Toronto...


In [6]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(Savee + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,angry,surrey-audiovisual-expressed-emotion-savee/ALL...
1,angry,surrey-audiovisual-expressed-emotion-savee/ALL...
2,angry,surrey-audiovisual-expressed-emotion-savee/ALL...
3,angry,surrey-audiovisual-expressed-emotion-savee/ALL...
4,angry,surrey-audiovisual-expressed-emotion-savee/ALL...


In [7]:
# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

,Emotions,Path
0,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
1,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
2,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
3,neutral,ravdess-emotional-speech-audio/Actor_01/03-01-...
4,calm,ravdess-emotional-speech-audio/Actor_01/03-01-...


In [8]:
def create_waveplot(data, sr, e):
    plt.figure(figsize=(10, 3))
    plt.title('Waveplot for audio with {} emotion'.format(e), size=15)
    librosa.display.waveshow(data, sr=sr)
    plt.show()

def create_spectrogram(data, sr, e):
    # stft function converts the data into short term fourier transform
    X = librosa.stft(data)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(12, 3))
    plt.title('Spectrogram for audio with {} emotion'.format(e), size=15)
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')   
    #librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar()

In [9]:
emotion='fear'
path = np.array(data_path.Path[data_path.Emotions==emotion])[1]
print(path)
data, sampling_rate = librosa.load(path)
Audio(path)

ravdess-emotional-speech-audio/Actor_01/03-01-06-01-01-02-01.wav


In [10]:
emotion='angry'
path = np.array(data_path.Path[data_path.Emotions==emotion])[1]
print(path)
data, sampling_rate = librosa.load(path)

Audio(path)

ravdess-emotional-speech-audio/Actor_01/03-01-05-01-01-02-01.wav


In [11]:
emotion='disgust'
path = np.array(data_path.Path[data_path.Emotions==emotion])[1]
print(path)
data, sampling_rate = librosa.load(path)

Audio(path)

ravdess-emotional-speech-audio/Actor_01/03-01-07-01-01-02-01.wav


In [12]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(y=data, rate=rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor)

# taking any example and checking for techniques.
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

In [20]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data,sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2)) # stacking vertically
    
    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch)
    result = np.vstack((result, res3)) # stacking vertically
    
    return result

In [21]:
print(path)

ravdess-emotional-speech-audio/Actor_01/03-01-01-01-01-02-01.wav


In [22]:

X, Y = [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    feature = get_features(path)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [23]:
len(X), len(Y), data_path.Path.shape

(36486, 36486, (12162,))

In [24]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv('features.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.321275,0.729664,0.750033,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,...,4.310903e-06,3.291511e-06,2.148075e-06,2.279739e-06,5.116493e-06,8.190282e-06,0.000007,0.000005,4.245834e-07,neutral
1,0.283117,0.763143,0.792962,0.770799,0.781775,0.777305,0.672709,0.666988,0.722528,0.753494,...,7.033042e-06,6.028629e-06,4.790136e-06,4.966464e-06,8.034008e-06,1.111317e-05,0.000010,0.000008,3.171771e-06,neutral
2,0.188256,0.622128,0.699219,0.753332,0.721223,0.701737,0.682353,0.662828,0.686490,0.733966,...,8.576332e-07,9.576413e-07,7.733594e-07,5.233102e-07,3.592796e-07,9.261687e-07,0.000002,0.000001,7.753984e-08,neutral
3,0.293566,0.673896,0.722096,0.723508,0.682302,0.680533,0.675352,0.628977,0.679179,0.707283,...,6.984504e-06,7.034949e-06,6.654923e-06,6.979548e-06,1.214236e-05,9.640183e-06,0.000011,0.000006,4.254087e-07,neutral
4,0.281349,0.762709,0.787711,0.756628,0.742797,0.766342,0.678706,0.654341,0.713586,0.742848,...,1.426636e-05,1.425237e-05,1.348651e-05,1.372349e-05,1.981722e-05,1.686377e-05,0.000019,0.000012,7.856675e-06,neutral


In [1]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values


NameError: name 'Features' is not defined

In [40]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
print(Y)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [41]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((27364, 162), (27364, 8), (9122, 162), (9122, 8))

In [42]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((27364, 162), (27364, 8), (9122, 162), (9122, 8))

In [43]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((27364, 162, 1), (27364, 8), (9122, 162, 1), (9122, 8))

In [44]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

# Add LSTM layer
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=8, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 162, 256)          1536      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 81, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 81, 256)           327936    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 41, 256)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 41, 128)           163968    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 21, 128)          0

In [29]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks=[rlrp])

Epoch 1/100
428/428 [==============================] - 45s 99ms/step - loss: 1.6635 - accuracy: 0.3197 - val_loss: 1.4109 - val_accuracy: 0.4130 - lr: 0.0010
Epoch 2/100
428/428 [==============================] - 43s 100ms/step - loss: 1.4162 - accuracy: 0.4306 - val_loss: 1.3053 - val_accuracy: 0.4819 - lr: 0.0010
Epoch 3/100
428/428 [==============================] - 43s 100ms/step - loss: 1.3156 - accuracy: 0.4749 - val_loss: 1.2407 - val_accuracy: 0.4925 - lr: 0.0010
Epoch 4/100
428/428 [==============================] - 42s 99ms/step - loss: 1.2636 - accuracy: 0.4975 - val_loss: 1.1994 - val_accuracy: 0.5202 - lr: 0.0010
Epoch 5/100
428/428 [==============================] - 42s 99ms/step - loss: 1.2224 - accuracy: 0.5126 - val_loss: 1.1815 - val_accuracy: 0.5308 - lr: 0.0010
Epoch 6/100
428/428 [==============================] - 42s 99ms/step - loss: 1.1905 - accuracy: 0.5288 - val_loss: 1.1530 - val_accuracy: 0.5324 - lr: 0.0010
Epoch 7/100
428/428 [=============================

In [30]:
print("Accuracy of our model on test data : " , model.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(100)]

train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']



286/286 [==============================] - 4s 15ms/step - loss: 1.7699 - accuracy: 0.6476
Accuracy of our model on test data :  64.75553512573242 %


In [ ]:
model.save('1minimodels/ser_lstm_model.h5')

In [31]:
# predicting on test data.

pred_test = model.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

286/286 [==============================] - 5s 15ms/step


In [44]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)

,Predicted Labels,Actual Labels
0,sad,disgust
1,disgust,disgust
2,angry,angry
3,happy,disgust
4,disgust,fear
5,disgust,fear
6,sad,happy
7,happy,happy
8,disgust,sad
9,neutral,sad


In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       angry       0.74      0.78      0.76      1438
        calm       0.73      0.80      0.77       137
     disgust       0.59      0.53      0.56      1468
        fear       0.61      0.60      0.61      1424
       happy       0.63      0.63      0.63      1462
     neutral       0.60      0.60      0.60      1310
         sad       0.62      0.66      0.64      1400
    surprise       0.84      0.87      0.86       483

    accuracy                           0.65      9122
   macro avg       0.67      0.68      0.68      9122
weighted avg       0.65      0.65      0.65      9122

